# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [10]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [4]:
def expected_return(P, rf, mkt_rp):
    P['ev'] = rf + mkt_rp * P['Beta']
    total = P['Amount Invested'].sum()
    P['Proportional Return'] = P['ev'] * P['Amount Invested'] / total
    return P['Proportional Return'].sum()
expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [21]:
def volatility(P, cor):
    total = P['Amount Invested'].sum()
    P['weight'] = P['Amount Invested'] * 1.0 / total
    ret = 0
    print(P)
    for x in range(len(P)):
        for y in range(len(P)):
            if x != y:
                ret += P['weight'].iloc[x] * P['weight'].iloc[y] * cor * P['Volatility'].iloc[x] * P['Volatility'].iloc[y]
            else:
                ret += (P['weight'].iloc[x] ** 2) * P['Volatility'].iloc[x] ** 2
    return ret ** 0.5
volatility(portfolio, -0.2)

   Beta  Volatility  Amount Invested     stdev  weight
0   1.2        0.25             4000  0.500000     0.4
1   1.4        0.40             6000  0.632456     0.6


0.24083189157584592

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [24]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Covariance between RF and any asset is always 0
    x_mkt = vol / mkt_vol
    return rf + x_mkt * mkt_rp
expected_return2(0.02, 0.05, 0.16, 0.2408)

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov($R_{i} - R_{f}, R_{mkt} – R_{f}$)/Var($R_{mkt}-R_{f}$). 

expected return: it should return output of 3225 rows × 3 columns

In [53]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    data = {
        "CUSIP" : [] ,
        "EXP_RETURN" : [],
        "BETA" : [],
    }
    
    # "2".zfill(2) --> "02"
    mkt_returns = pd.read_csv(factors_path)
    mkt_returns['date'] = mkt_returns['date'].apply(str)
    mkt_returns['Mkt-RF'] = mkt_returns['Mkt-RF'] / 100.0
    mkt_returns['RF'] = mkt_returns['RF'] / 100.0
    stock_returns = pd.read_csv(return_path)
    
    stock_returns['date'] = stock_returns['year'].apply(str) + stock_returns['month'].apply(lambda x: str(x).zfill(2))
    
    for group in stock_returns.groupby(['CUSIP'], group_keys = False):
        stock = group[-1]
        
        data['CUSIP'] += (stock['CUSIP'].iloc[0]),
        data['EXP_RETURN'] += (stock['RET'].mean()),
        
        join = stock.merge(mkt_returns, left_on = 'date', right_on = 'date', how = 'inner')
        join['RET-RF'] = join['RET'] - join['RF']
        cov = join['RET-RF'].cov(join['Mkt-RF'])
        data['BETA'] += (cov / join['Mkt-RF'].var()),
    
    return pd.DataFrame(data)

CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV")

C:\Users\Alan PC\AppData\Local\Temp\ipykernel_16560\3889049699.py:18: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group in stock_returns.groupby(['CUSIP'], group_keys = False):


CUSIP  EXP_RETURN      BETA
0     00081T10    0.016252  2.591073
1     00108M10    0.011101  0.741057
2     00130H10    0.016152  1.457100
3     00163U10    0.015538  0.975530
4     00182C10    0.014772  1.406085
...        ...         ...       ...
3220   9959910    0.009933  0.763107
3221   9972410    0.012866  1.280548
3222   9972510    0.017177  1.065298
3223   9984910    0.009006  1.876418
3224   9985510    0.015699  1.166459

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [54]:
capm_df = CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV")

C:\Users\Alan PC\AppData\Local\Temp\ipykernel_16560\3889049699.py:18: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group in stock_returns.groupby(['CUSIP'], group_keys = False):


In [56]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    return np.polyfit(data['BETA'], data['EXP_RETURN'], 1)[1]

slope(capm_df)

[0.00477324 0.00798159]


0.007981588120535637